In [9]:
import datetime
from qiskit_finance.data_providers import RandomDataProvider
import numpy as np

In [10]:
n_qubits = 4
seed = 123

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(n_qubits)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2016, 1, 1),
    end=datetime.datetime(2016, 1, 30),
    seed=seed,
)
data.run()

mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

In [11]:
def QUBO_from_portfolio(cov, mean, q, B, t):
    # cov: n x n covariance numpy array
    # mean: numpy array of means
    n = cov.shape[0]
    R = np.diag(mean)
    S = np.ones((n, n)) - 2 * B * np.diag(np.ones(n))

    Q = q * cov - R + t * S
    return Q

In [12]:
q = 0.5
budget = 3  # Note that in this example, there are 4 assets, but a budget of only 3
penalty = 3

Q = QUBO_from_portfolio(sigma, mu, q, budget, penalty)

In [13]:
# Brutely search over classical results for comparison before we run QAOA
# the results are sorted with cost
states = []
for i in range(2**n_qubits):
    a = f"{bin(i)[2:]:0>{n_qubits}}"
    n_ones = 0
    for j in a:
        if j == '1':
            n_ones += 1
    if True:#n_ones != budget:
        states.append(a)
cost_dict = {}
for selection in states:
    x = np.array([int(bit) for bit in selection])
    cost_dict[selection] = np.dot(x, np.dot(Q, x))
cost_sorted = dict(sorted(cost_dict.items(), key=lambda item: item[1]))
print("\n-------------------------------------")
print("    selection\t  |\t  cost")
print("-------------------------------------")
for k, v in cost_sorted.items():
    print("%10s\t  |\t%.4f" % (k, v))
print("-------------------------------------")


-------------------------------------
    selection	  |	  cost
-------------------------------------
      1011	  |	-27.0150
      1101	  |	-27.0139
      1110	  |	-27.0130
      0111	  |	-27.0000
      1001	  |	-24.0149
      1010	  |	-24.0140
      1111	  |	-24.0139
      1100	  |	-24.0130
      0011	  |	-24.0010
      0101	  |	-23.9998
      0110	  |	-23.9992
      1000	  |	-15.0140
      0001	  |	-15.0008
      0010	  |	-15.0001
      0100	  |	-14.9991
      0000	  |	0.0000
-------------------------------------
